In [1]:
import numpy as np

In [3]:
class Node(object):
    def __init__(self,inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.value = None
        self.outbound_nodes = []
        for node in inbound_nodes:
            node.outbound_nodes.append(self)
    
    def forward():
        raise NotImplemented

In [11]:
class Input(Node):
    """
    While it may be strange to consider an input a node when
    an input is only an individual node in a node, for the sake
    of simpler code we'll still use Node as the base class.

    Think of Input as collating many individual input nodes into
    a Node.
    """
    def __init__(self):
        # An Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Node.__init__(self)
    
    def forward(self):
        # Do nothing because nothing is calculated.
        pass

In [12]:
class Linear(Node):
    def __init__(self,X,W,b):
        # Notice the ordering of the input nodes passed to the
        # Node constructor.
        Node.__init__(self,[X,W,b])
    
    def forward(self):
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2].value
        self.value = 0
        self.value += np.dot(inputs,weights)+bias

In [54]:
def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """
    input_nodes = [n for n in feed_dict.keys()]
    print("input_nodes: ",len(input_nodes))
    G = {}
    nodes = [n for n in input_nodes]
    print("nodes: ",len(nodes))
    while len(nodes) > 0:
        n = nodes.pop(0)
        print("n_len: ",type(n))
        if n not in G:
            G[n] = {'in': set(),'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in':set(),'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
    
    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()
        
        if isinstance(n,Input):
            n.value = feed_dict[n]
        
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
        print(len(L))
    return L
                

In [55]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted Nodes.

    Arguments:

        `output_node`: A Node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: a topologically sorted list of nodes.

    Returns the output node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [56]:
X, W, b = Input(), Input(), Input()
f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])
feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)
print(output)

input_nodes:  3
nodes:  3
n_len:  <class '__main__.Input'>
n_len:  <class '__main__.Input'>
n_len:  <class '__main__.Input'>
n_len:  <class '__main__.Linear'>
n_len:  <class '__main__.Linear'>
n_len:  <class '__main__.Linear'>
1
2
3
4
[[-9.  4.]
 [-9.  4.]]
